In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [06:13<00:00,  4.79it/s]  


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [02:06<00:00,  8.07it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [8]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [9]:
trainer.train()

epoch=0.126953125, loss=5.051429748535156:   2%|▏         | 65/4096 [00:28<1:38:57,  1.47s/it]

epoch=0.125, eval_loss=4.97737780213356


epoch=0.251953125, loss=4.219053268432617:   3%|▎         | 129/4096 [00:55<1:54:42,  1.74s/it]

epoch=0.25, eval_loss=4.169926807284355


epoch=0.376953125, loss=3.9263572692871094:   5%|▍         | 193/4096 [01:24<1:52:14,  1.73s/it]

...

epoch=7.75, eval_loss=3.168650910258293


epoch=7.875, loss=3.0055019855499268:  98%|█████████▊| 4032/4096 [54:41<01:33,  1.47s/it]      

epoch=7.875, eval_loss=3.167494721710682


epoch=8.0, loss=3.109633445739746: 100%|██████████| 4096/4096 [56:20<00:00,  1.21it/s]         

epoch=8.0, eval_loss=3.1640994176268578


# Оцениваем качество и проверяем жадную и случайную генерацию

In [10]:
trainer.evaluate()

3.1640994176268578

In [11]:
generate(model, tokenizer, temperature=0)

'М и наиболее эффективность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможность, возможнос

In [12]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Сек в этом выяснувины проблемой наские задачи, ссоры или природный утрального и задуманшего: например, подсказсливинтересные и прям вторжения или корот, если вы худебных людей и март, особенно красдивать свои потребностью, если вы неудачуется неверной жизни.'